```
The MIT License (MIT)

Copyright (c) 2020 NVIDIA

Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
the Software, and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

```

# Interacting with TRITON Inference Server

Walthrough a generic pipeline for:
- Using client libraries to run inference on models on TRITON sever
- Modifying model configs

#### Environment
**1 - Client**
All steps executed using **NGC Pytorch Docker (v 20.06)**
* TRITON Client libraries installed within container

**2 - Server**
using **NGC tritonserver Docker (20.06-v1)**

![TRITON Workflow](./NB_images/TRITON_image.png)

Start a new terminal and start the TRITON server by running the following command

`start_triton_server.sh`

This simulates a "Server" running in a separate docker container

In [5]:
import argparse
import numpy as np
import os
import json
from builtins import range
import tensorrtserver.api.model_config_pb2 as model_config
from tensorrtserver.api import *
from dataset_utils import *

In [6]:
list_files('./trtismodelrepo/v1/model_cxr_trt/')

/
    config.pbtxt
    labels.txt
1/
    model.plan


In [7]:
!cat './trtismodelrepo/v1/model_cxr_trt/config.pbtxt'

name: "model_cxr_trt"
platform: "tensorrt_plan"
max_batch_size : 96
input [
  {
    name: "input"
    data_type: TYPE_FP32
    format: FORMAT_NCHW
    dims: [ 3, 224,224 ]
  }
]
output [
  {
    name: "output"
    data_type: TYPE_FP32
    dims: [ 14 ]
    label_filename: "labels.txt"
  }
]
instance_group [
  {
    count: 2
    kind: KIND_GPU
  }
]
dynamic_batching {
  preferred_batch_size: [ 80 ]
}



## Check status of Inference Server/Specific Model

In [13]:
# !curl localhost:8000/api/status

In [8]:
url = 'localhost:8000'
protocol = ProtocolType.HTTP

health_ctx = ServerHealthContext(url, protocol, verbose=True)
print("Live: {}".format(health_ctx.is_live()))
print("Ready: {}".format(health_ctx.is_ready()))

Live: True
Ready: True


In [10]:
model_name = "model_cxr_trt"
# Create a status context and get server status
status_ctx = ServerStatusContext(url, protocol, model_name, verbose=True)
print("Status for model {}".format(model_name))
print(status_ctx.get_server_status())

Status for model model_cxr_trt
id: "inference:0"
version: "1.14.0"
uptime_ns: 241180253490
model_status {
  key: "model_cxr_trt"
  value {
    config {
      name: "model_cxr_trt"
      platform: "tensorrt_plan"
      version_policy {
        latest {
          num_versions: 1
        }
      }
      max_batch_size: 96
      input {
        name: "input"
        data_type: TYPE_FP32
        format: FORMAT_NCHW
        dims: 3
        dims: 224
        dims: 224
      }
      output {
        name: "output"
        data_type: TYPE_FP32
        dims: 14
        label_filename: "labels.txt"
      }
      instance_group {
        name: "model_cxr_trt_0"
        count: 1
        gpus: 0
        kind: KIND_GPU
      }
      default_model_filename: "model.plan"
      dynamic_batching {
        preferred_batch_size: 80
      }
      optimization {
        input_pinned_memory {
          enable: true
        }
        output_pinned_memory {
          enable: true
        }
      }
    }
    ver

In [11]:
ctx = ServerStatusContext(url, protocol, model_name, True)
server_status = ctx.get_server_status()

if model_name not in server_status.model_status:
    raise Exception("unable to get status for '" + model_name + "'")

status = server_status.model_status[model_name]
config = status.config

In [12]:
def parse_model(url, protocol, model_name, batch_size, verbose=False):
    """
    Check the configuration of a model to make sure it meets the
    requirements
    """
    ctx = ServerStatusContext(url, protocol, model_name, verbose)
    server_status = ctx.get_server_status()

    if model_name not in server_status.model_status:
        raise Exception("unable to get status for '" + model_name + "'")

    status = server_status.model_status[model_name]
    config = status.config

    if len(config.input) != 1:
        raise Exception("expecting 1 input, got {}".format(len(config.input)))
    if len(config.output) != 1:
        raise Exception("expecting 1 output, got {}".format(len(config.output)))

    input = config.input[0]
    output = config.output[0]
    
    # Model specifying maximum batch size of 0 indicates that batching
    # is not supported and so the input tensors do not expect an "N"
    # dimension (and 'batch_size' should be 1 so that only a single
    # image instance is inferred at a time).
    max_batch_size = config.max_batch_size
    if max_batch_size == 0:
        if batch_size != 1:
            raise Exception("batching not supported for model '" + model_name + "'")
    else: # max_batch_size > 0
        if batch_size > max_batch_size:
            raise Exception("expecting batch size <= {} for model {}".format(max_batch_size, model_name))

#     Model input must have 3 dims, either CHW or HWC
    if len(input.dims) != 3:
        raise Exception(
            "expecting input to have 3 dimensions, model '{}' input has {}".format(
                model_name, len(input.dims)))

    if input.format == model_config.ModelInput.FORMAT_NHWC:
        h = input.dims[0]
        w = input.dims[1]
        c = input.dims[2]
    else:
        c = input.dims[0]
        h = input.dims[1]
        w = input.dims[2]

    return (input.name, output.name, c, h, w, input.format, input.data_type)

In [13]:
model_version = -1
batch_size = 80

input_name, output_name, c, h, w, format, dtype = parse_model(url, protocol, model_name, batch_size, verbose=True)

## Setup Data for inference

In [14]:
from dataset_utils import *

def get_batched_image_input(bs):
    all_inputs = []
    dataset_json = 'CXR14_dataset.json'
    with open(dataset_json) as json_file:
        data = json.load(json_file)
    
    test_loader = get_test_loader_TTA(data, batch_size=bs)

    # Using Test Time Augmentation    
    for i, (input, target) in enumerate(test_loader):
        bs, n_crops, c, h, w = input.size()
        input = input.view(-1, c, h, w)

        all_inputs.append(input)

    return all_inputs, data['labels'][0]

bs = 8 
im_tensor, labels = get_batched_image_input(bs)

## Setup inference on images

In [16]:
def triton_inferer(ctx, input_name, output_name, batch):
    batch = [batch[i].cpu().numpy().astype(np.float32) for i in range(0,batch.shape[0]) ]
    input_dict = { input_name : batch }
    output_dict = { output_name : (InferContext.ResultFormat.RAW)}
    results = ctx.run(
        inputs=input_dict, 
        outputs=output_dict, 
        batch_size=len(batch)
    )
    return results[output_name]

def process_results_cxr(out):
    outputs = torch.cat([torch.tensor(np.atleast_2d(im)) for im in out])
    output = torch.nn.functional.softmax(outputs, dim=1).cpu()
    output = output.view(8, 10, -1).mean(1).numpy()
    top5 = np.argsort(output)[:,-5:][:,::-1]


    for i in range(8):
        t5 = top5[i]
        op = output[i]
        txt = 'Image {} -- '.format(i+1)
        for c, v in zip(t5, op[t5]):
            txt = txt + f"{labels[str(c)]}: {100*v:.1f}% "
        print(txt)

In [17]:
model_name = "model_cxr_trt"
model_version = -1
ctx = InferContext(url, protocol, model_name, model_version, verbose=False)
out = triton_inferer(ctx, input_name, output_name, im_tensor[0])
process_results_cxr(out)

Image 1 -- Cardiomegaly: 90.4% Effusion: 5.7% Emphysema: 2.0% Nodule: 0.5% Pleural_Thickening: 0.5% 
Image 2 -- Cardiomegaly: 91.4% Emphysema: 3.2% Effusion: 2.7% Infiltration: 1.0% Nodule: 0.5% 
Image 3 -- Cardiomegaly: 88.1% Effusion: 7.4% Emphysema: 1.6% Infiltration: 0.8% Atelectasis: 0.6% 
Image 4 -- Infiltration: 33.5% Nodule: 32.7% Emphysema: 9.1% Effusion: 8.8% Pleural_Thickening: 5.6% 
Image 5 -- Nodule: 51.6% Mass: 40.5% Infiltration: 4.2% Consolidation: 1.7% Atelectasis: 1.4% 
Image 6 -- Infiltration: 22.7% Nodule: 22.0% Atelectasis: 19.1% Pleural_Thickening: 8.7% Effusion: 6.9% 
Image 7 -- Infiltration: 39.7% Nodule: 30.7% Atelectasis: 7.3% Consolidation: 6.3% Pleural_Thickening: 4.5% 
Image 8 -- Infiltration: 34.8% Pneumothorax: 25.8% Atelectasis: 16.2% Consolidation: 9.7% Nodule: 3.9% 


In [18]:
model_name = "model_cxr_trt_fp16"
ctx = InferContext(url, protocol, model_name, model_version, verbose=False)
out = triton_inferer(ctx, input_name, output_name, im_tensor[0])
process_results_cxr(out)

Image 1 -- Cardiomegaly: 90.3% Effusion: 5.7% Emphysema: 2.0% Nodule: 0.5% Pleural_Thickening: 0.5% 
Image 2 -- Cardiomegaly: 91.3% Emphysema: 3.2% Effusion: 2.8% Infiltration: 1.0% Nodule: 0.5% 
Image 3 -- Cardiomegaly: 88.0% Effusion: 7.5% Emphysema: 1.6% Infiltration: 0.8% Atelectasis: 0.7% 
Image 4 -- Infiltration: 33.6% Nodule: 32.6% Emphysema: 9.0% Effusion: 8.8% Pleural_Thickening: 5.6% 
Image 5 -- Nodule: 51.5% Mass: 40.6% Infiltration: 4.2% Consolidation: 1.7% Atelectasis: 1.4% 
Image 6 -- Infiltration: 22.7% Nodule: 22.1% Atelectasis: 18.9% Pleural_Thickening: 8.6% Effusion: 6.8% 
Image 7 -- Infiltration: 39.6% Nodule: 30.7% Atelectasis: 7.3% Consolidation: 6.3% Pleural_Thickening: 4.6% 
Image 8 -- Infiltration: 34.7% Pneumothorax: 26.0% Atelectasis: 16.1% Consolidation: 9.7% Nodule: 3.9% 


In [19]:
model_name = "model_cxr_onnx"
ctx = InferContext(url, protocol, model_name, model_version, verbose=False)
out = triton_inferer(ctx, input_name, output_name, im_tensor[0])
process_results_cxr(out)

Image 1 -- Cardiomegaly: 90.4% Effusion: 5.7% Emphysema: 2.0% Nodule: 0.5% Pleural_Thickening: 0.5% 
Image 2 -- Cardiomegaly: 91.4% Emphysema: 3.2% Effusion: 2.7% Infiltration: 1.0% Nodule: 0.5% 
Image 3 -- Cardiomegaly: 88.1% Effusion: 7.4% Emphysema: 1.6% Infiltration: 0.8% Atelectasis: 0.6% 
Image 4 -- Infiltration: 33.5% Nodule: 32.7% Emphysema: 9.1% Effusion: 8.8% Pleural_Thickening: 5.6% 
Image 5 -- Nodule: 51.6% Mass: 40.5% Infiltration: 4.2% Consolidation: 1.7% Atelectasis: 1.4% 
Image 6 -- Infiltration: 22.7% Nodule: 22.0% Atelectasis: 19.1% Pleural_Thickening: 8.7% Effusion: 6.9% 
Image 7 -- Infiltration: 39.7% Nodule: 30.7% Atelectasis: 7.3% Consolidation: 6.3% Pleural_Thickening: 4.5% 
Image 8 -- Infiltration: 34.8% Pneumothorax: 25.8% Atelectasis: 16.2% Consolidation: 9.7% Nodule: 3.9% 


In [23]:
model_name = "model_cxr_onnx"
# Create a status context and get server status
status_ctx = ServerStatusContext(url, protocol, model_name, verbose=True)
print("Status for model {}".format(model_name))
print(status_ctx.get_server_status())

Status for model model_cxr_onnx
id: "inference:0"
version: "1.14.0"
uptime_ns: 329289074009
model_status {
  key: "model_cxr_onnx"
  value {
    config {
      name: "model_cxr_onnx"
      platform: "onnxruntime_onnx"
      version_policy {
        latest {
          num_versions: 1
        }
      }
      max_batch_size: 80
      input {
        name: "input"
        data_type: TYPE_FP32
        format: FORMAT_NCHW
        dims: 3
        dims: 224
        dims: 224
      }
      output {
        name: "output"
        data_type: TYPE_FP32
        dims: 14
        label_filename: "labels.txt"
      }
      instance_group {
        name: "model_cxr_onnx"
        count: 1
        gpus: 0
        kind: KIND_GPU
      }
      default_model_filename: "model.onnx"
      dynamic_batching {
        preferred_batch_size: 80
      }
      optimization {
        input_pinned_memory {
          enable: true
        }
        output_pinned_memory {
          enable: true
        }
      }
    }
  

In [21]:
import time

In [24]:
counter = 1
ctx = InferContext(url, protocol, model_name, model_version, verbose=False)
while counter < 10000:
    out = triton_inferer(ctx, input_name, output_name, im_tensor[0])
    if counter % 100 == 0: print('Iteration {}'.format(counter))
    time.sleep(2)
    counter += 1

Iteration 100


KeyboardInterrupt: 